In [2]:
import pandas as pd
import numpy as np

Two datasets - 

CDP based GHG emissions data - https://data.cdp.net/Companies/2013-Global-500-Emissions-and-Response-Status/marp-zazk
S&P500 - Balance Sheet and Income Data - https://simfin.com/data/bulk

For faster processing, i combined the three data sets in excel, however the process can be easily replicated in python using merge function.Broadly, the steps for pre-processing are- 

1. Filtering for US based companies and any blanks in Scope 1/Scope 2 data. 
2. Introduce a new variable ticker in data downloaded from SimFin which is Ticker symbol + "US"
3. Combine both these data, with merge on ticker name. 


In [17]:
#loading the dataset
path = r"C:\Users\Gaurav\Downloads\sp_data.xlsx"

data = pd.read_excel(path)
data = data.dropna(axis=0)
data.drop(["Unnamed: 15","Unnamed: 16"],axis=1,inplace=True)
data.head()

,Company Name,Account Number,Ticker Symbol,ISIN,Disclosure Score,Performance Band,Permission,Response Status,Scope 1 (metric tonnes CO2e),Scope 2 (metric tonnes CO2e),Ticker,Reporting,Total Assets,Total Equity,Total Liabilities,Revenue,Gross Profit,Research & Development,Operating Income (Loss),Net Income (Common)
0,CSX Corporation,4120,CSX US,US1264081035,95.0,A,Public,AQ*,5268905.0,300170.0,CSX US,2016.0,35414.0,11694.0,23720.0,11069.0,4690.0,0.0,3389.0,1714.0
2,3M Company,285,MMM US,US88579Y1010,70.0,D,Public,AQ*,4540000.0,2230000.0,MMM US,2016.0,32906.0,10343.0,22563.0,30109.0,15069.0,-1735.0,7112.0,5050.0
5,"The Home Depot, Inc.",8526,HD US,US4370761029,99.0,A-,Public,AQ*,250224.0,2529646.0,HD US,2015.0,39946.0,9322.0,30624.0,88519.0,30265.0,0.0,11774.0,7009.0
6,Las Vegas Sands Corporation,10494,LVS US,US5178341070,98.0,A-,Public,AQ*,225157.0,821527.0,LVS US,2016.0,20469.0,7496.0,12973.0,11271.0,5888.0,-9.0,2581.0,1679.0
16,FedEx Corporation,6287,FDX US,US31428X1063,80.0,B,Public,AQ*,14602697.0,959109.0,FDX US,2016.0,45959.0,13784.0,32175.0,50365.0,14457.0,0.0,4575.0,1820.0


CDP Data points 

Performance Band - Corresponds to a particular Scoring Level (Disclosure, Awareness, Management, Leadership) which demonstrates the company’s level of environmental stewardship and your actions and approaches in managing climate change. 



Ref - https://www.esg.adec-innovations.com/assets/img/whitepaper/pdf/4-steps-to-improving-your-cdp-performance-whitepaper-12072017_2.pdf



In [21]:
# We define a combined Emission = Scope 1 + Scope 2

data["Total Emissions"] = data["Scope 1 (metric tonnes CO2e)"]+data["Scope 2 (metric tonnes CO2e)"]
data["Emissions / Assets"] = data["Total Emissions"]/data["Total Assets"]
data["Emissions / Equity"] = data["Total Emissions"]/data["Total Equity"]

data.head()

,Company Name,Account Number,Ticker Symbol,ISIN,Disclosure Score,Performance Band,Permission,Response Status,Scope 1 (metric tonnes CO2e),Scope 2 (metric tonnes CO2e),...,Total Assets,Total Equity,Total Liabilities,Revenue,Gross Profit,Research & Development,Operating Income (Loss),Net Income (Common),Total Emissions,Emissions / Assets
0,CSX Corporation,4120,CSX US,US1264081035,95.0,A,Public,AQ*,5268905.0,300170.0,...,35414.0,11694.0,23720.0,11069.0,4690.0,0.0,3389.0,1714.0,5569075.0,157.256311
2,3M Company,285,MMM US,US88579Y1010,70.0,D,Public,AQ*,4540000.0,2230000.0,...,32906.0,10343.0,22563.0,30109.0,15069.0,-1735.0,7112.0,5050.0,6770000.0,205.737555
5,"The Home Depot, Inc.",8526,HD US,US4370761029,99.0,A-,Public,AQ*,250224.0,2529646.0,...,39946.0,9322.0,30624.0,88519.0,30265.0,0.0,11774.0,7009.0,2779870.0,69.590697
6,Las Vegas Sands Corporation,10494,LVS US,US5178341070,98.0,A-,Public,AQ*,225157.0,821527.0,...,20469.0,7496.0,12973.0,11271.0,5888.0,-9.0,2581.0,1679.0,1046684.0,51.135082
16,FedEx Corporation,6287,FDX US,US31428X1063,80.0,B,Public,AQ*,14602697.0,959109.0,...,45959.0,13784.0,32175.0,50365.0,14457.0,0.0,4575.0,1820.0,15561806.0,338.601928


### Exclusion

Retail investors can define their exclusion criteria - for eg-  no exposure to Tobacco / Arms manufacturing / Oil

Other Criteria could be related Universe [for eg: Only US] / Disclosure Score [for eg: only top 50 percentile] 

In [33]:
# sorting for Performance bands -D, E - 

exclusion_performance_band = ['D','E']
list1 = data.loc[~data['Performance Band'].isin(exclusion)]

#Selecting companies in top 50 percentile of median score 

exclusion_disclosure_score = data["Disclosure Score"].median()
list1 = data.loc[data["Disclosure Score"]>exclusion_disclosure_score]

list1.describe()

,Account Number,Disclosure Score,Scope 1 (metric tonnes CO2e),Scope 2 (metric tonnes CO2e),Reporting,Total Assets,Total Equity,Total Liabilities,Revenue,Gross Profit,Research & Development,Operating Income (Loss),Net Income (Common),Total Emissions,Emissions / Assets
count,47.000000,47.000000,4.700000e+01,4.700000e+01,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,4.700000e+01,47.000000
mean,9863.510638,95.255319,4.483897e+06,2.227239e+06,2015.893617,76351.875617,27964.104872,48387.770745,55187.571234,19591.406979,-1586.073383,6468.644702,4717.918128,6.711136e+06,133.739246
std,6958.480587,3.384543,1.014771e+07,3.371997e+06,0.311661,79769.605137,36747.312555,51260.276321,79352.939052,23508.249817,3221.778489,7405.868593,5497.858961,1.176246e+07,248.989039
min,333.000000,90.000000,2.349520e+03,2.299500e+04,2015.000000,5453.000000,-10900.000000,5272.411000,4762.000000,2326.400000,-13948.000000,-6216.000000,-6173.000000,2.573900e+04,0.784401
25%,3842.000000,92.000000,1.035485e+05,4.174450e+05,2016.000000,22298.900000,6211.450000,13455.000000,11359.900000,5023.595000,-1325.500000,1813.350000,1447.000000,6.808155e+05,16.220737
50%,8526.000000,96.000000,3.362580e+05,8.537000e+05,2016.000000,40262.000000,12409.000000,30624.000000,26685.000000,10490.000000,0.000000,3837.000000,3363.000000,1.696000e+06,42.282606
75%,15284.000000,98.000000,4.470694e+06,2.152931e+06,2016.000000,104683.000000,33959.038000,60179.000000,67344.500000,27116.000000,0.000000,9245.000000,6747.000000,6.884704e+06,152.829864
max,23195.000000,100.000000,5.855922e+07,1.558114e+07,2016.000000,403821.000000,146722.000000,279711.000000,482130.000000,121146.000000,0.000000,27188.000000,20539.000000,6.240854e+07,1405.003106


In [ ]:

#creating a emission impact score 
list1["Emission Impact"] 
